In [1]:
import requests
import time
#from time import strftime
import datetime
import numpy as np
import re
import xmltodict
import pprint# as pp
#import process_command
#print(dir(process_command))

In [2]:
class timer:
    import timer
    def __init__(self,tickLength=60,debug=False):
        self.span = tickLength
        self.debug=debug
        self.goTick=False

    def tick(self, debug=False):
        now=time.time()
        if not self.goTick:
            self.starttime = now+0.
            self.goTick=True
            self.lastTick = now+0.
        sleepTime = span - ((now - starttime) % span)
        nextTick = now + sleepTime
        if nextTick > self.lastTick + self.span*1.1:
            if self.debug:
                print("  - timer: missed a tick")
        if self.debug:
            print("  - timer: sleepTime = "+str(sleepTime))
        time.sleep(sleepTime)    
        self.lastTick = time.time()
    
    def tickList_set(self,aList, epsilon = 0.1):
        self.tickList = aList
        self.tickListPointer=0
        self.epsilon=epsilon
    
    def tickList_go(self): # use this method in a loop before doing something! First call starts counter.
        if self.tickListPointer==0:
            self.tickList_startTime=time.time()
        differenceReal = time.time() - self.tickList_startTime
        differencePlanned = self.tickList[self.tickListPointer]
            
        if differenceReal < differencePlanned:
            sleepTime = differencePlanned-differenceReal 
            if self.debug:
                print("  - timer: sleepTime =",sleepTime)
            time.sleep(sleepTime)
        elif not self.tickListPointer==0:
            if self.debug:
                print("  - timer: warning. took to long time. Moving immidiatelly")

        if self.tickListPointer==len(self.tickList)-1:
            if self.debug:
                print("  - timer: tick list ends after next action.")
            return False
        self.tickListPointer+=1
        return True

In [3]:
tickList=[0,6,11,15,18,20,21,23,26,30,35,41]
t = timer(debug=False)
t.tickList_set(tickList)
go=True
starter=time.time()
while(go):
    go=t.tickList_go()
    print("new loop, time:",time.time()-starter)
    [i*i for i in range(21000000)]
    

new loop, time: 0.0
new loop, time: 6.016844749450684
new loop, time: 11.004231929779053
new loop, time: 15.016154766082764
new loop, time: 18.002872467041016
new loop, time: 20.833758115768433
new loop, time: 23.550990104675293
new loop, time: 26.271601676940918
new loop, time: 28.97834348678589
new loop, time: 31.690468072891235
new loop, time: 35.01791977882385
new loop, time: 41.01428842544556


In [4]:
def openWebsite(adress):
    r = requests.get(adress)
    return r.text

In [5]:
def prettyPrint(text):
    pp = pprint.PrettyPrinter(indent=1)
    pp.pprint(text)

In [6]:
def process_command_vbb(command, param):  
    def furtherParamString(param):#,listToDelete):
        ret=""
        for key in param.keys():
            ret+= "&"+str(key)+"="+str(param[key])
        return ret
    beforeId=command
    behindId=furtherParamString(param)
    return {"beforeId":beforeId, "behindId":behindId}

In [7]:
def createAdress_vbb(command, param):
    baseurl="http://demo.hafas.de/openapi/vbb-proxy/"
    commands = process_command_vbb(command, param)
    accessId ="felix-fauer-8b71-1705950c2589"
    adress = baseurl + commands['beforeId'] + "?accessId=" + accessId + commands['behindId']
    print(adress)
    return adress

In [8]:
def responseToDict(response):
    return xmltodict.parse(response)

In [9]:
def getData(command, param):
    response_xml = openWebsite(createAdress_vbb(command, param))
    return responseToDict(response_xml)    

In [10]:
def nameToExt(name):
    response = getData('location.name', {"input":name,"maxNo":1})
    return response['LocationList']['StopLocation']['@extId']

In [11]:
def short_departureAt(name, maxNo=3):
    ext=nameToExt(name)
    command, param ='departureBoard', {'extId':ext,'maxJourneys':maxNo}
    response = getData(command, param)
    return response

In [12]:
def nice_nextXDeparturesAt(name, maxNo=3):
    response = short_departureAt(name, maxNo)
    print("Next departures at "+str(response['DepartureBoard']['Departure'][0]['@stop'])+":\n")
    print("   Line        Destination                     Time                  (intern data)")
    print("   ----        -----------                     ----")
    for dep in response['DepartureBoard']['Departure']:
        if "@rtTime" in dep.keys():
            if dep['@time'][0:5]!= dep['@rtTime'][0:5]:
                print("   "+dep['@name'].lstrip().ljust(8)+"    "+dep['@direction'].ljust(30)[0:30]+"  "+
                      dep['@time'][0:5]+" ("+dep['@rtTime'][0:5]+")"+"".ljust(9)+dep['JourneyDetailRef']['@ref'])
            else: 
                print("   "+dep['@name'].lstrip().ljust(8)+"    "+dep['@direction'].ljust(30)[0:30]+"  "+
                      dep['@rtTime'][0:5]+"       "+"".ljust(10)+dep['JourneyDetailRef']['@ref'])
        else:
            print("   "+dep['@name'].lstrip().ljust(8)+"    "+dep['@direction'].ljust(30)[0:30]+"  "+
                  dep['@time'][0:5]+" (?)    "+"".ljust(9)+dep['JourneyDetailRef']['@ref'])


In [13]:
nice_nextXDeparturesAt("Tauernallee/Säntisstraße",5)
#nice_nextXDeparturesAt("Messe Nord",20)

http://demo.hafas.de/openapi/vbb-proxy/location.name?accessId=felix-fauer-8b71-1705950c2589&input=Tauernallee/Säntisstraße&maxNo=1
http://demo.hafas.de/openapi/vbb-proxy/departureBoard?accessId=felix-fauer-8b71-1705950c2589&extId=900070401&maxJourneys=5
Next departures at Tauernallee/Säntisstr. (Berlin):

   Line        Destination                     Time                  (intern data)
   ----        -----------                     ----
   M76         U Walther-Schreiber-Platz       23:57 (?)             1|25382|6|86|19022022
   M76         S Lichtenrade                   00:02 (?)             1|25341|4|86|19022022
   179         U Alt-Mariendorf                00:06 (?)             1|13663|14|86|19022022
   X76         U Alt-Mariendorf                00:06 (?)             1|29032|9|86|19022022
   179         Buckow, Gerlinger Str.          00:09 (?)             1|13627|15|86|19022022


In [14]:
#command, param ='location.nearbystops', {'originCoordLat':52.50798,'originCoordLong':13.38,'maxNo':2}
#command, param ='departureBoard', {'extId':900070401,'maxNo':1}
command, param ='location.name', {"input":"Tauernallee/Säntisstraße","maxNo":2}
#command, param ='journeyDetail', {"id":"1|25024|8|86|18052019"}


In [15]:
response = getData(command, param)
#prettyPrint(response)
#print(short_departureAt("Tauernallee/Säntisstraße"))
#prettyPrint(short_departureAt("Zoolo"))


http://demo.hafas.de/openapi/vbb-proxy/location.name?accessId=felix-fauer-8b71-1705950c2589&input=Tauernallee/Säntisstraße&maxNo=2
